In [1]:
import os

os.getcwd()
os.chdir("../")
os.getcwd()

'/home/mobo/Documents/Ineuron/Live_Class/DL/DL_live_notes/CNN/deepCNNClassifier'

In [18]:
from deepClassifier.constants import *
from deepClassifier import logger

In [3]:
from deepClassifier.utils import read_yaml, create_directories

In [4]:
from ensure import ensure_annotations

In [4]:
read_yaml("configs/config.yaml")

{'artifact_root': 'artifacts',
 'data_ingestion': {'root_dir': 'artifact_root/data_ingestion',
  'source_dir': 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip',
  'local_data_dir': 'artifacts/data_ingestion/data.zip',
  'unzipped_dir': 'artifacts/data_ingestion'}}

In [71]:
import sys

class DeepClassifierException(Exception):
    def __init__(
        self, 
        error_message: Exception,
        error_detail: sys
        ):
        
        # super(DeepClassifierException, self).__init__(error_message)
        self.error_message = \
            DeepClassifierException.get_detailed_error_message(
                error_message=error_message,
                error_detail=error_detail
            )
        super(DeepClassifierException, self).__init__(self.error_message)
        
        
        
    @staticmethod
    def get_detailed_error_message(error_message: Exception,
                                   error_detail: sys):
        _, _, exec_tb = error_detail.exc_info()
        exception_line_no = exec_tb.tb_frame.f_lineno
        try_block_no = exec_tb.tb_lineno
        file_name = exec_tb.tb_frame.f_code.co_filename
        print(file_name, try_block_no, exception_line_no)
        error_message = f"""
        Error occured in script:
        [{file_name}]
        raised by line: [{exception_line_no}]
        on line: [{try_block_no}]
        """
        return error_message
    


    
# a,b = 2, 0

# try:
#     if b == 0: from e
#         raise Exception("can not divide this!")
# except Exception as e:
#     _, _, exec_tb = sys.exc_info()
#     exception_line_no = exec_tb.tb_frame.f_lineno
#     try_block_no = exec_tb.tb_lineno
#     file_name = exec_tb.tb_frame.f_code.co_filename
#     print(file_name, try_block_no, exception_line_no)
#     error_message = f"""
#     Error occured in script:
#     [{file_name}]
#     raised by line: [{exception_line_no}]
#     on line: [{try_block_no}]
#     """
#     print(error_message)



In [72]:
def div(a,b):

    try:
        if b == 0:
            # raise Exception as e
        
            return a/b
    except Exception as e:
        raise DeepClassifierException(e, sys) from e

In [73]:
div(2,0)

/tmp/ipykernel_88077/1023788321.py 7 9


DeepClassifierException: 
        Error occured in script:
        [/tmp/ipykernel_88077/1023788321.py]
        raised by line: [9]
        on line: [7]
        

In [14]:
from collections import namedtuple

DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_dir",
    "local_data_dir",
    "unzipped_dir"
])

In [38]:
## solution without ConfigBox


class ConfigurationManager:
    
    # @ensure_annotations
    def __init__(
        self, 
        config_filepath: str=CONFIG_FILE_PATH,
        params_filepath: str=PARAMS_FILE_PATH) -> None:
        self.config_details = read_yaml(config_filepath)
        self.params_details = read_yaml(params_filepath)
        self.artifact_root = self.config_details["artifact_root"]
        
    @ensure_annotations
    def data_ingestion_config(self) -> DataIngestionConfig:
        root_dir = self.config_details["data_ingestion"]["root_dir"]
        source_dir = self.config_details["data_ingestion"]["source_dir"]
        local_data_dir = self.config_details["data_ingestion"]["local_data_dir"]
        unzipped_dir = self.config_details["data_ingestion"]["unzipped_dir"]
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=root_dir,
            source_dir=source_dir,
            local_data_dir=local_data_dir,
            unzipped_dir=unzipped_dir
        )
        
        return data_ingestion_config
        

In [40]:
config = ConfigurationManager(config_filepath=CONFIG_FILE_PATH)
config.data_ingestion_config()

DataIngestionConfig(root_dir='artifact_root/data_ingestion', source_dir='https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', local_data_dir='artifacts/data_ingestion/data.zip', unzipped_dir='artifacts/data_ingestion')

In [26]:
# solution with ConfigBox

from box import ConfigBox

class ConfigurationManager:
    
    def __init__(
        self, 
        config_file_path=CONFIG_FILE_PATH, 
        params_file_path=PARAMS_FILE_PATH):
        self.config_file_path = ConfigBox(read_yaml(config_file_path))
        self.artifacts = self.config_file_path.artifact_root
        # self.params_file_path = ConfigBox(read_yaml(params_file_path))
        
    def data_ingestion_config(self) -> DataIngestionConfig:
        try:
            data_ingestion = self.config_file_path.data_ingestion
            root_dir = data_ingestion.root_dir
            source_dir = data_ingestion.source_dir
            local_data_dir = data_ingestion.local_data_dir
            unzipped_dir = data_ingestion.unzipped_dir
            
            data_ingestion_config = DataIngestionConfig(
                root_dir=root_dir,
                source_dir=source_dir,
                local_data_dir=local_data_dir,
                unzipped_dir=unzipped_dir
            )
            logger.info(f"DataIngestionConfig setup: \n{data_ingestion_config}")
            return data_ingestion_config
        
        except Exception as e:
            raise e
        

In [27]:
config = ConfigurationManager()
config.data_ingestion_config()

[2022-09-16 10:35:30,871: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-09-16 10:35:30,954: INFO: 480393586]: DataIngestionConfig setup: 
DataIngestionConfig(root_dir='artifact_root/data_ingestion', source_dir='https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', local_data_dir='artifacts/data_ingestion/data.zip', unzipped_dir='artifacts/data_ingestion')


DataIngestionConfig(root_dir='artifact_root/data_ingestion', source_dir='https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', local_data_dir='artifacts/data_ingestion/data.zip', unzipped_dir='artifacts/data_ingestion')

In [20]:
class Person:
    def __init__(self, name: str) -> None:
        self.name = name
        
    @staticmethod
    def age(age: int) -> str:
        return f"your age is: {age}"
    
    def __str__(self) -> str:
        return f"'{self.name}'"
    
    def __repr__(self) -> str:
        return f"Person(name='{self.name}')"

In [21]:
p = Person("paul")
p.age(20)
print(p)

'paul'
